## Time evolution

Another main use of dynamite is computing the time evolution of quantum states under a Hamiltonian. For an example, let's watch a state evolve under the Heisenberg model:

$$H = \sum_{i,j} \vec{S}_i \cdot \vec{S}_j$$

where $\vec{S} = (S^x, S^y, S^z)$.

**Important**: Don't forget that the spin operators have an extra factor of 1/2 relative to the Pauli matrices!

In [ ]:
from dynamite import config
config.L = 14

In [ ]:
from dynamite.operators import sigmax, sigmay, sigmaz, index_sum

def heisenberg():
    paulis = [sigmax, sigmay, sigmaz]
    
    # 0.25 to account for the factors of 1/2 in each spin operator!
    return index_sum(sum(0.25*p(0)*p(1) for p in paulis))

H = heisenberg()
H

To perform an evolution, we need to start with a state. Let's initialize a state with one domain wall:

In [ ]:
from dynamite.states import State

half = config.L//2
initial_state = State(state='U'*half + 'D'*half)

The $S^z$ expectation values across the chain:

In [ ]:
from matplotlib import pyplot as plt

# for performance reasons, its a good idea to pre-compute the Sz operators
# if we're going to be re-using them a lot
Sz_ops = [0.5*sigmaz(i) for i in range(config.L)]

def plot_sigma_z(state):
    z_vals = [Szi.expectation(state) for Szi in Sz_ops]
    plt.plot(z_vals)
    plt.xlabel('Spin index')
    plt.ylabel('Z magnetization')
    
plot_sigma_z(initial_state)

Now let's evolve for a little bit of time, and see how the magnetization changes. The `.evolve()` function computes $e^{-iHt} | \psi \rangle$:

In [ ]:
final_state = H.evolve(initial_state, t=3)
plot_sigma_z(final_state)

Beautiful---we see that as the evolution proceeds, the domain wall is relaxing. 

Generally, dynamite is used to track various quantities as time evolution proceeds. For example, let's watch the magnetization of each spoin evolve over time. The compute time it takes to perform a time evolution is roughly proportional to the evolution time, so if you are taking intermediate measurements it's most efficient to evolve in a series of small $\Delta t$, starting where you left off in the previous step:

In [ ]:
total_evolution_time = 10
delta_t = 0.2

n_time_steps = int(total_evolution_time/delta_t)

times = [0]
vals = [[Szi.expectation(initial_state) for Szi in Sz_ops]]

result = initial_state.copy()  # pre-allocating the result vector improves performance
for i in range(n_time_steps):
    times.append(times[-1] + delta_t)
    H.evolve(initial_state, t=delta_t, result=result)
    vals.append([Szi.expectation(result) for Szi in Sz_ops])
    
    # swap initial state and result
    # because the previous result becomes the next initial state
    initial_state, result = result, initial_state
    
# it may take a few seconds to run...

In [ ]:
# plot the trajectory of the magnetization for each spin
for spin_vals in zip(*vals):
    plt.plot(times, spin_vals)

plt.xlabel('Time')
plt.ylabel(r'Magnetization $\langle S^Z_i \rangle$')

It's kind of beautiful! If you'd like, try similarly plotting the evolution of the half-chain entanglement entropy over time. You can compute the entanglement entropy of a state via the `State.entanglement_entropy` function (see the documentation [here](https://dynamite.readthedocs.io/en/latest/dynamite.states.html#dynamite.states.State.entanglement_entropy)).

This kind of evolution can be easily adapted to piecewise time-dependent Hamiltonians, by just adjusting the Hamiltonian that performs the evolution. For example, try implementing a Floquet evolution: try adding to the Heisenberg model a z-field that flips polarity with every period of T=1. This is most easily achieved by building two Hamiltonians (for each piece of the volution) and switching back and forth which one you use for the evolution step.

In [ ]:
# Exercise: implement the Floquet evolution of a Heisenberg model with a Z-field that flips every 1 unit of time.


## Up next

[Continue to notebook 5](5-Subspaces.ipynb) to learn how to work in smaller subspaces to make computations more efficient.